## tf-garf

This is an implementation of "[Gaussian Activated Radiance Fields](https://arxiv.org/abs/2204.05735)" by Chng et al. in TensorFlow 2.

<a href='https://github.com/laura-a-n-n'><img src='https://github.com/favicon.ico' alt='GitHub' style='display:inline;width:2em; height:2em' /></a>
<a href='https://twitter.com/laura_a_n_n'><img src='https://img.icons8.com/color/344/twitter--v1.png' alt='GitHub' style='display:inline; width:2.2em; height:2.2em' /></a>
<a href='https://lauraann.dev/'><img src='https://img.icons8.com/ios/344/domain.png' alt='GitHub' style='display:inline; width:2em; height:2em' /></a> *made with love by Laura Ann*

### Data

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from lib.data import load_data

from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog

def select_files(b):
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    b.files = filedialog.askdirectory()
    
    tf.print(b.files)
    display(load_button)

def process(b):
    global sel_button
    
    tf.print('Loading data...')
    b.data = load_data(sel_button.files)
    tf.print('Data loaded.')

sel_button = widgets.Button(description='Select folder...')
sel_button.on_click(select_files)

load_button = widgets.Button(description='Load data')
load_button.on_click(process)

display(sel_button)

### Train

In [ ]:
''' training !!! '''

from model.garf import GaussianRadianceField
from train import train

model = GaussianRadianceField(load_button.data, num_samples=128)
model.compile()

def standard_train(b):
    train(model, 200000, batch_size=64, val_idx='rand')

train_button = widgets.Button(description='Train')
train_button.on_click(standard_train)
display(train_button)

### Render

In [ ]:
model.load('pretrain/flowers')

In [ ]:
for i in range(num_img):
    xx, yy = model.render(i)
    tf.print(tf.image.psnr(xx, yy, 1.))

In [ ]:
xx, yy = model.render(0)
tf.image.psnr(xx, yy, 1.)